In [ ]:
# data_clean_book_sample.ipynb

import pandas as pd
from pathlib import Path

BASE = Path("../../raw_data/books")
books_path = BASE / "books.csv"
book_tags_path = BASE / "book_tags.csv"
tags_path = BASE / "tags.csv"

print("📘 Loading data...")
books = pd.read_csv(books_path)
book_tags = pd.read_csv(book_tags_path)
tags = pd.read_csv(tags_path)

print(f"books.csv: {books.shape}, book_tags.csv: {book_tags.shape}, tags.csv: {tags.shape}")


print("\n🔍 Columns:")
print("books.csv:", books.columns.tolist())
print("book_tags.csv:", book_tags.columns.tolist())
print("tags.csv:", tags.columns.tolist())


if "book_id" in book_tags.columns:
    book_tags = book_tags.rename(columns={"book_id": "goodreads_book_id"})


if "book_id" not in books.columns and "id" in books.columns:
    books = books.rename(columns={"id": "book_id"})


print("\n🔗 Merging book_tags and tags...")
book_tags = book_tags.merge(tags, on="tag_id", how="left")


book_tag_grouped = (
    book_tags.groupby("goodreads_book_id")["tag_name"]
    .apply(lambda x: ", ".join(x.head(10)))
    .reset_index()
    .rename(columns={"tag_name": "tag_list"})
)

print(f"✅ Created tag_list for {len(book_tag_grouped)} books")

print("\n🔗 Merging tags into books.csv...")
books = books.merge(book_tag_grouped, left_on="book_id", right_on="goodreads_book_id", how="left")


books["original_title"] = books["original_title"].fillna(books["title"])
books["authors"] = books["authors"].fillna("Unknown")
books["average_rating"] = pd.to_numeric(books["average_rating"], errors="coerce")
books["ratings_count"] = pd.to_numeric(books["ratings_count"], errors="coerce")
books["original_publication_year"] = pd.to_numeric(books["original_publication_year"], errors="coerce")

print(f"✅ Combined shape: {books.shape}")

print("\n🎯 Filtering high-rated, popular books...")
books_filt = books[
    (books["average_rating"] >= 4.0) &
    (books["ratings_count"] >= 5000) &
    (books["original_publication_year"].between(1900, 2025))
].copy()

print(f"📚 Remaining after filter: {len(books_filt)}")

C = books_filt["average_rating"].mean()
m = books_filt["ratings_count"].quantile(0.75)
v = books_filt["ratings_count"]
R = books_filt["average_rating"]
books_filt["weighted_score"] = (v/(v+m))*R + (m/(v+m))*C

books_filt = books_filt.sort_values("weighted_score", ascending=False)


book_sample = books_filt.head(100).copy()

out_path = Path("../../data_clean/book/book_sample_100.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)

keep_cols = [
    "book_id","title","original_title","authors","original_publication_year",
    "average_rating","ratings_count","tag_list","image_url","small_image_url"
]
keep_cols = [c for c in keep_cols if c in book_sample.columns]
book_sample[keep_cols].to_csv(out_path, index=False, encoding="utf-8")

print(f"\n✅ Saved {len(book_sample)} books → {out_path}")
print("sample:")
print(book_sample[["title","authors","average_rating","tag_list"]].head(5))


📘 Loading data...


FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\raw_data\\books\\books.csv'

In [2]:
%conda install pandas

3 channel Terms of Service accepted
Channels:
 - defaults
Platform: win-64
Solving environment: done

## Package Plan ##

  environment location: c:\Users\tadyh\miniconda3\envs\dsan6700

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.4.2           |   py39hc99e966_0         129 KB
    mkl-service-2.4.0          |   py39h827c3e9_2          66 KB
    mkl_fft-1.3.11             |   py39h827c3e9_0         167 KB
    mkl_random-1.2.8           |   py39hc64d2fc_0         257 KB
    numexpr-2.10.1             |   py39h4cd664f_0         177 KB
    numpy-2.0.1                |   py39h055cbcc_1          11 KB
    numpy-base-2.0.1           |   py39h65a83cf_1         9.0 MB
    pandas-2.3.3               |   py39ha5e6156_0        11.0 MB
    python-tzdata-2025.2       |     pyhd3eb1b0_0         141 KB
    pytz-2025.2                |   py39haa95

C:\Users\tadyh\miniconda3\Lib\site-packages\conda\base\context.py:211: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.9. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(


==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c defaults conda


